In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torchvision.transforms as transforms
import pandas as pd
import ast
torch.manual_seed(1234)

# Data Preprocessing to include label and shift it one place up

In [3]:
#Preprocessing
nepse_data = pd.read_csv("data_set_ready_for_training.csv")
nepse_data["news"] = nepse_data["news"].apply(ast.literal_eval)
#nepse_data.set_index("date", inplace=True)

In [4]:
nepse_data.head(10)


,closing_price,news,momentum,EMA,MACD,RSI,ROI,ATR,williams,CCI,UO
0,975.0,"[[0.2306930273771286, 0.792356550693512, 0.642...",-87.0,1087.863864,-58.947313,0.149378,1.126154,106.0,0.820755,997.666667,0.154372
1,954.0,"[[0.31689509749412537, 0.8638337850570679, 0.6...",-2.0,1067.269422,-65.253876,0.250000,1.122642,38.0,1.000000,966.666667,0.105707
2,917.0,"[[0.31150949001312256, 0.7647785544395447, 0.6...",-75.0,1044.151049,-72.401798,0.480000,1.135878,75.0,1.000000,942.000000,0.069584
3,903.0,"[[0.1868913620710373, 0.7952941060066223, 0.61...",-87.0,1022.435502,-78.293327,0.000000,1.120598,87.0,1.000000,932.000000,0.068895
4,885.0,"[[0.3303453326225281, 0.8576347231864929, 0.68...",-90.0,1001.291578,-83.452392,0.000000,1.108927,90.0,1.000000,915.000000,0.108076
5,897.0,"[[0.20605844259262085, 0.858686089515686, 0.64...",-57.0,985.246720,-85.586305,0.133333,1.062542,69.0,0.826087,912.000000,0.117757
6,947.0,"[[0.2074199914932251, 0.7867178320884705, 0.59...",30.0,979.362609,-82.295166,0.898551,0.994298,62.0,0.000000,926.333333,0.204783
7,910.0,"[[0.42128807306289673, 0.7513155937194824, 0.6...",7.0,968.691438,-81.729667,0.898551,1.029670,62.0,0.596774,914.000000,0.203104
8,925.0,"[[0.3032321333885193, 0.9355986714363098, 0.52...",40.0,961.969678,-79.158862,1.400000,1.005730,62.0,0.354839,919.000000,0.194369
9,900.0,"[[0.17515438795089722, 0.8498166799545288, 0.7...",3.0,952.435882,-78.236491,1.241935,1.023667,50.0,0.940000,914.666667,0.205341


In [5]:
nepse_data['Label'] = nepse_data['closing_price'].rolling(window=2).apply(lambda x:  1 if x[1]>x[0] else 0 )
nepse_data.Label = nepse_data.Label.shift(-1)
nepse_data = nepse_data[:-1]


# From here on, data is prepared to be fed to pytorch network

The dataset is prepared by inheriting Dataset class, and DataLoader class is used for batching. 


In [6]:
#DataLoader class
class NepseDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.features = dataset.loc[:, 'momentum':'CCI'].as_matrix()
        self.label =  dataset['Label'].as_matrix()
        self.news = dataset['news'].as_matrix()
        self.transform = transform
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        tech_indicators = self.features[idx,:]
        label = self.label[idx]
        news = self.news[idx]
        sample = {'news': news,'tech_indicators': tech_indicators, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        return sample
    
       
        

In [7]:
class ToTensor(object):

    def __call__(self, sample):
        news, tech_indicators, label = sample['news'], sample['tech_indicators'], sample['label']

        return {'news': torch.FloatTensor(news),
                'tech_indicators': torch.from_numpy(tech_indicators).float(),
                'label': int(label)}

In [8]:
def append_zeros(news):
    padding_dims = 30 - len(news)
    for _ in range(padding_dims):
        news.append([0]*100)
    return news
def truncate(news):
    return news[0:30]

In [9]:
#print(nepse_data.news)
for i in range(len(nepse_data.news)):
    
    if len(nepse_data.news[i])<30:
        nepse_data.news[i] = append_zeros(nepse_data.news[i])
    elif len(nepse_data.news[i])>30:
        nepse_data.news[i] = truncate(nepse_data.news[i])
        

test_data = NepseDataset(nepse_data, transform=ToTensor())
print(len(test_data))
test_data[0]['news'].shape

c:\users\pc\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
c:\users\pc\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


1377


torch.Size([30, 100])

In [10]:
data = DataLoader(test_data, batch_size=10)
for i, sample in enumerate(data):
    print(sample['news'].shape)
    break

torch.Size([10, 30, 100])


# Our Neural Network
 The network consists of CNN which extracts features from news, the output of which is concatenated with the technical indicators and fed to lstm. The output is then fed to softmax to predict the rise or fall in nepse for the following day.

In [323]:

class StockNN(nn.Module):
    def __init__(self, filter_sizes, drop_prob, embedding_dim, length_of_features, n_hidden_lstm=256, n_layers=1):
        super(StockNN, self).__init__()
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = 1, 
                                              out_channels = 2, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        self.lstm = nn.LSTM(length_of_features, n_hidden_lstm, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.linear = nn.Linear(n_hidden_lstm, 2)
        
        
    def forward(self,x,h):
        
        x_cnn = x['news']
        x_cnn = x_cnn.reshape(x_cnn.shape[0],1,x_cnn.shape[1], x_cnn.shape[2])
        conved = [F.relu(conv(x_cnn)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        stock_features = self.dropout(torch.cat(pooled, dim = 1).squeeze())
        
        stock_features = torch.cat([stock_features,x['tech_indicators']], dim=1)
        out, (h,c) = self.lstm(stock_features.view(stock_features.shape[0],1,-1),h)
        hidden_2_risefall = self.dropout(self.linear(out.view(out.shape[0],-1)))
        rise_fall = F.softmax(hidden_2_risefall, dim=1)
        
        return rise_fall, (h,c)
    
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(1, n_seqs, 256).zero_(),
                weight.new(1, n_seqs, 256).zero_())


# Train Validation Split

In [324]:
def train_validate_set(data,val_frac=0.1):
    m = data.shape[0]
    val_idx = int(m*(1-val_frac))
    test_data, val_data = NepseDataset(data.head(val_idx),ToTensor()), NepseDataset(data.tail(m - val_idx),ToTensor())
    return test_data, val_data

td, vd = train_validate_set(nepse_data) 
print(len(td), len(vd))

1239 138


# Training the StockNN


In [325]:
#Have to check this one
def train(NN, data, val_frac=0.2, max_epochs=10, batch_size=10, learning_rate=0.001):
    NN.train()
    opt = torch.optim.Adam(NN.parameters(), lr=learning_rate)
    
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    train_data, validate_data = train_validate_set(data,val_frac)
    dataloader = DataLoader(train_data, batch_size=batch_size)
    for e in range(max_epochs):
        h = NN.init_hidden(batch_size)
        
        for i_batch, batched_sample in enumerate(dataloader):
            h = tuple([each.data for each in h])
            
            opt.zero_grad()
            xtrain = {key: batched_sample.get(key) for key in batched_sample if key == 'news' or key == 'tech_indicators'}
            out, h= NN(xtrain,h)
            loss = criterion(out,batched_sample['label'])
            loss.backward()
            
            opt.step()
            print(e, i_batch, loss.item())


  

In [326]:
d1 = torch.Tensor([[1,2,3],[5,9,6]])
d2 = torch.Tensor([[4,5,6,7],[0,8,9,4]])
torch.cat([d1,d2],dim=1)
np.multiply(*d1.shape)

6

In [327]:
stock_net = StockNN([2,3,4], 0.5, 100, 14)

In [328]:

train(stock_net, nepse_data, max_epochs= 10, batch_size=7)

0 0 0.675051212310791
0 1 0.7055670022964478
0 2 0.7940192818641663
0 3 0.772091269493103
0 4 0.6749516725540161
0 5 0.6558800935745239
0 6 0.6484292149543762
0 7 0.6340871453285217
0 8 0.6128209829330444
0 9 0.8438916802406311
0 10 nan
0 11 nan
0 12 nan
0 13 nan
0 14 nan
0 15 nan
0 16 nan
0 17 nan
0 18 nan
0 19 nan
0 20 nan
0 21 nan
0 22 nan
0 23 nan
0 24 nan
0 25 nan
0 26 nan
0 27 nan
0 28 nan
0 29 nan
0 30 nan
0 31 nan
0 32 nan
0 33 nan
0 34 nan
0 35 nan
0 36 nan
0 37 nan
0 38 nan
0 39 nan
0 40 nan
0 41 nan
0 42 nan
0 43 nan
0 44 nan
0 45 nan
0 46 nan
0 47 nan
0 48 nan
0 49 nan
0 50 nan
0 51 nan
0 52 nan
0 53 nan
0 54 nan
0 55 nan
0 56 nan
0 57 nan
0 58 nan
0 59 nan
0 60 nan
0 61 nan
0 62 nan
0 63 nan
0 64 nan
0 65 nan
0 66 nan
0 67 nan
0 68 nan
0 69 nan
0 70 nan
0 71 nan
0 72 nan
0 73 nan
0 74 nan
0 75 nan
0 76 nan
0 77 nan
0 78 nan
0 79 nan
0 80 nan
0 81 nan
0 82 nan
0 83 nan
0 84 nan
0 85 nan
0 86 nan
0 87 nan
0 88 nan
0 89 nan
0 90 nan
0 91 nan
0 92 nan
0 93 nan
0 94 nan
0 95 na

RuntimeError: Expected hidden[0] size (1, 2, 256), got (1, 7, 256)